# DeepCluster: Learning Good Representations for Trace Clustering

In [1]:
import numpy as np
import pandas as pd

from replearn.eventlog import EventLog

from replearn.embedding_predict import EmbeddingPredict
from replearn.autoencoder import AutoencoderRepresentation
from replearn.doc2vec import Doc2VecRepresentation

from replearn.clustering import Clustering

## Load event log

In [2]:
# event log configuration
event_log_path = '../logs/'
file_name = 'huge_500_10_20_5_1_1-0.0-1.json.gz'

case_attributes = None # auto-detect attributes
event_attributes = ['concept:name', 'user'] # use activity name and user
true_cluster_label = 'cluster'

# load file
event_log = EventLog(file_name, case_attributes=case_attributes, event_attributes=event_attributes, true_cluster_label=true_cluster_label)
event_log.load(event_log_path + '/' + file_name, False)
event_log.preprocess()

## Representation Learning

In [3]:
# hyperparameters
n_epochs = 10
n_batch_size = 64
n_clusters = 5

vector_size = 32

### LSTM / GRU

In [4]:
# init and train LSTM
predictor = EmbeddingPredict(event_log)
predictor.build_model(embedding_dim=vector_size, gru_dim=vector_size, rnn='LSTM')
predictor.fit(epochs=n_epochs, batch_size=n_batch_size, verbose=True)

Epoch 1/10
8/8 [==============================] - 0s 16ms/step - loss: 24.7798 - dense_loss: 2.4499 - dense_1_loss: 2.3969 - dense_2_loss: 2.5101 - dense_3_loss: 2.3853 - dense_4_loss: 2.9272 - dense_5_loss: 2.4240 - dense_6_loss: 2.5699 - dense_7_loss: 2.4159 - dense_8_loss: 2.2719 - dense_9_loss: 2.4285 - dense_accuracy: 0.1120 - dense_1_accuracy: 0.1180 - dense_2_accuracy: 0.1160 - dense_3_accuracy: 0.1940 - dense_4_accuracy: 0.0520 - dense_5_accuracy: 0.1560 - dense_6_accuracy: 0.1680 - dense_7_accuracy: 0.1220 - dense_8_accuracy: 0.1740 - dense_9_accuracy: 0.2140
Epoch 2/10
8/8 [==============================] - 0s 17ms/step - loss: 21.2913 - dense_loss: 2.0486 - dense_1_loss: 2.0386 - dense_2_loss: 2.1503 - dense_3_loss: 2.0466 - dense_4_loss: 2.5123 - dense_5_loss: 2.0348 - dense_6_loss: 2.2137 - dense_7_loss: 2.1412 - dense_8_loss: 2.0291 - dense_9_loss: 2.0761 - dense_accuracy: 0.2520 - dense_1_accuracy: 0.2720 - dense_2_accuracy: 0.2240 - dense_3_accuracy: 0.2760 - dense_4_ac

In [5]:
# get feature vector
pred_model, feature_vector, embedding_vector = predictor.predict()

### Clustering

In [6]:
# cluster feature vector
cluster_analysis = Clustering(event_log)
cluster_analysis.cluster(feature_vector, 'agglomerative', n_clusters, 'cosine')

cluster_result = cluster_analysis.evaluate()

In [7]:
print('Adjusted Rand Index: ' + str(cluster_result[0]))
print('Normalized Mutual Information: ' + str(cluster_result[1]))
print('F1-BCubed: ' + str(cluster_result[2]))

Adjusted Rand Index: 1.0
Normalized Mutual Information: 1.0
F1-BCubed: 1.0
